# Working with Spark DataFrames
When it comes to all the strucutured APIs in Spark, the DataFrames API is probably the most commonly used way to 
interact with Spark functionality because its most familiar with most users. In this notebook, we will dive deeper into the DataFrames 
API and explore the following topics:
- Creating Spark DataFrames with Schema and the advantage of using schemas
- Manipulating DataFrames (e.g., adding colummns, renaming columns, changing data types etc
- Query DataFrames. How do you filter data?
- Performing common data analysis tasks on DataFrames (e.g., grouping data, applying functions)
- Applying user defined functions in Spark DataaFrames
- Switching between DataFrames and other data structures (e.g., Pandas, RDD's etc)

## Python setup

In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
from functools import wraps
import time
from IPython.display import Image
from datetime import datetime

## Inputs setup
Lets provide paths to input files we will use. 
Its a good practice to create these as global variables. Also, use Python module ```Path``` from pathlib to manage file paths. 

In [2]:
# Altenatively, you can put a full-path to wheree your data is located like below
# DATA_DIR = Path(full-path-folder-where-you-are-keeping-data)
DATA_DIR = Path().cwd().parents[0].joinpath("DATA")


# Activity_log_raw
ACTIV_LOG = DATA_DIR.joinpath("raw", "activity_log_raw.csv")

# path to hh_data.txt
HH_DATA = DATA_DIR.joinpath("raw", "hh_data.txt")

## Composition of a Spark DataFrame
In Spark, a DataFrame object consists of [Row](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Row.html) objects and [Column](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.html) objects. Concretely, each row of a Spark DataFrame  is an instance of the ```pyspark.sql.Row``` while each column is an instance of the ```pyspark.sql.Column``` class. We will look at  each of these classes in detail.

## Creating DataFrames
There are three main ways to create a Spark DataFrame as follows:
1. From Python objects
2. External data sources
3. Other Spark objects

### Schemas
Also, when creating DataFrames, you have the option to use a schema or not. A schema in Spark defines the column names and associated data types for a DataFrame. Most often, schemas come into play when you are reading structured data from an external data source. When a schema is not used, Spark has to infer the data type which can slow your application if you have a massive  dataset. Although schemas are more of DBMS language but they offer several advantages when dealing with large datasets:
- Spark doesnt have to infer data types, so you get speed benefits.
- Without a schema, Spark creates a separate job just to read a large portion of your file to ascertain the schema, which for a large data file can be expensive and time-consuming. As such, defining a schema will avoid this.
- You can detect errors early if data doesn’t match the schema.
#### Defining Schemas
- Programmatically using Spark DataTypes 
- Using Data Definition Language (DDLs)

### Spark DataFrame from Python objects
Given a list of data (in most cases its going to be a nested list), how do we convert it into a Spark DataFrame. There are several depending on the input Pytho objects. Here are some:
1. Define a schema for the new dataframe. The schema helps Spark understand the column data types
2. Convert the regular Python objects into ```Rows``` which we can the use to create a DataFrame

In [4]:
# initialize SparkSession
spark = SparkSession.builder.master("local[*]").appName("DataFrameFromPythonObj").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 14:09:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/14 14:09:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/14 14:09:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/14 14:09:53 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/14 14:09:53 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [ ]:
# The data 
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter","LinkedIn"]],
           [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
    "LinkedIn"]],
           [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
    "twitter", "FB", "LinkedIn"]],
           [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,
    ["twitter", "FB"]],
           [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
    "twitter", "FB", "LinkedIn"]],
           [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,
    ["twitter", "LinkedIn"]]
          ]
# examine the data 
data[0]

In [ ]:
# Define schema for our data using DDL, in most cases whe 
schema_twitter = "num INT, first_name STRING, last_name STRING, url STRING, date STRING, some_num INT, tweet STRING"

In [ ]:
# Create a DataFrame using the schema defined above
sdf_python_obj_schema = spark.createDataFrame(data, schema=schema_twitter)
print(sdf_python_obj_schema.printSchema())

In [ ]:
# Convert list of items into Spark Rows
spark_rows = []

for item in data:
    row = Row(num=item[0], first_name=item[1], last_name=item[2], 
              url=item[3], date=item[4], some_num=item[5], tweet=item[6])
    spark_rows.append(row)

In [ ]:
sdf_from_rows = spark.createDataFrame(spark_rows)

In [ ]:
sdf_from_rows.show()

### Rows
A row in Spark is a generic Row object, containing one or more columns. Each column may be of the same data type (e.g., integer or string), or they can have different types (integer, string, map, array, etc.). 
Because Row is an object in Spark and an ordered collection of fields, you can instantiate a Row the same way we instantiate any object. Consequently, you can collect Row objects in a list and create a Spark DataFrame.

In [ ]:
# import row class
from pyspark import Row

first_five_rows = sdf.head(5)
type(first_five_rows[0])

# create row 
row = Row(name="Alice", age=11)

# A list of rows 
rows = [Row(name="Matei Zaharia", state="CA"), Row(name="Reynold Xin", state="CA")]

# Create a DataFrame from Row objects
spark_df_from_rows = spark.createDataFrame(rows)
spark_df_from_rows.show()

**EXERCISE-1:** Creating a Spark DataFrame with Rows. Please complete the function below and call it.

In [ ]:
def convert_json_to_spark_with_rows(json_file):
    # Load json into pandas DataFrame
    
    # create  a list to hold all Row objects
    rows = YOUR CODE
    for idx, row in df.iterrows():
        # get lon and lat from the coord column using indexing, dict key access
        x = row['coord']['lon']       
        y = row['coord']['lat']
        # create the Row object here 
        srow = YOUR CODE
        
        # append this row object to the list
        YOUR CODE
    
    # When creating Spark DataFrame this way, its better to use schema to avoid troubles
    # create a schema for this data here, use DOUBLE as data type for lon and lat
    schema = YOUR CODE
    
    # use spark.createDataFrame() here
    # if yiu get errors, use the option verifySchema=False
    spark_df = YOUR CODE
    
    # use show() statement to show the DataFrame
    # use show() with print to ensure we see the outputs
    YOUR CODE

In [ ]:
jsonfile = str(DATA_DIR.joinpath("raw", "city.list.json")
#convert_json_to_spark_with_rows(jsonfile)
df.show()

**EXERCISE-2: READ CSV WITH SCHEMA**
1. Use Spark documentation on how to read from file with a define schema. 
Note, the schema is what we arleady defined above. The data above has been saved as ```blog_simple_dataset.csv```. Read it as a Spark DataFrame with schema. Answer this question in the next cell.
2. Define schema for the ```activity_raw_data.csv``` use string for the datetime column
3. Load the dataset with and without schema using the functions defined below. Compare the loading times. Answer this question by completing the functions defined below and calling them.

In [ ]:
def timefn(fn):
    """
    Function for recording running time of a function
    """
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print("@timefn:" + fn.__name__ + " took " + str(t2 - t1) + " seconds")
        return result
    return measure_time

In [ ]:
@timefn
def load_with_schema(large_csv):
    # define the schema here using DDL
    # you can load part of the file with pandas (just a few rows)
    # to remind yourself of the data types
    schema = "`SID` INT, `ACTIVITY_ID` INT, `Last` STRING, `ACTIVITY_TIME` STRING,`STATUS` STRING"
    spark = SparkSession.builder.master("local[*]").appName("ReadWithChema").getOrCreate()
    # Now read the data 
    sdf = spark.read.schema(schema).csv(large_csv)
    
    print(sdf.show())

In [ ]:
@timefn
def load_without_schema(large_csv):
    spark = SparkSession.builder.master("local[*]").appName("DataFrameFromPythonObj").getOrCreate()
    sdf = spark.read.csv(large_csv, header=True)
    print(sdf.show())

In [ ]:
load_with_schema(str(ACTIV_LOG))

In [ ]:
load_without_schema(str(ACTIV_LOG))

<div class="alert alert-info"> 
The difference shown above will of course become signficant and consequential when the number of columns and size of data increase. As such =, although not required, in some use cases, you can use schema to improve performance.
</div>

### Spark DataFrame from external data sources
The most common way (which we have already seen) is to load data from exteernal data sources and 
Spark supports numerous data stores. Spark reads data  through the ```DataFrameReaderobject```. Please look at the documeentation [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameReader.csv.html) to see all data sources that the Spark  ```DataFrameReaderobject``` supports.

## Manipulating Spark DataFrames 

### Columns and Expressions in  DataFrames
In Spark DataFrames, columns behave like pandas DataFrames in several ways but they also behave different. You can list all the columns by their names, and you can perform operations on their values using relational or computational expressions. 
- [Column](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.html) is the name of the object, which has many import methods such as describe  while [col()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.col.html) is a standard built-in function that returns a Column.

We need to use the col() and expr() function available in pyspark,sql.functions() for many operations such as:
- Add, rename columns
- Subset data based on columns
- Access columns to compute stats on them
- Access columns to compute operations on them such as sorting

In [ ]:
# Lets use Activity_log_raw.csv for trying out manipulations
sdf = spark.read.csv(str(ACTIV_LOG), header=True)

### Selecting columns and rows 
How to select a single column and/or a subset of rows. We use ```select()``` to pick one or multipe. 
However, some of the same indexing we use in Pandas work, 

In [ ]:
# Find unnique values in column 'P21'
# Note that you have to use colleect() or show() 
# to bring results to driver and see, other this is a transformation
sdf2 = sdf.select('ACTIVITY_TIME', 'STATUS')
sdf2.show()

In [ ]:
# Use of the Pandas like indexing seem to also be supported like this 
# but just experiment with it because not everything is supported 
sdf_two_cols = sdf[['ACTIVITY_TIME', 'STATUS']]
sdf_two_cols.show(5)

### Add a new column using expr and col
In order to add a new column in a Spark DataFrame, we use the ```DataFrame.withColumn(new_col_name, expression_to_compute_new_col)```. 
For that, we need to import the ```col``` and ```expr``` functions from the ```pyspark.sql.functions``` module.

In [ ]:
sdf.show(5)

In [ ]:
# use expr
sdf2 = sdf.withColumn("status_S", (expr('STATUS = "S"')))
sdf2.show()

In [ ]:
# use the col function which I prefer over the expr col("Hits")
sdf2 = sdf.withColumn("new_col", col("STATUS") == 'S')
sdf2.show(10)

**EXERCISE-3:**

1. Check  if these statements: df.select(expr("ACTIVITY_TIME")).show(2), df.select(col("ACTIVITY_TIME")).show(2)
and df.select("ACTIVITY_TIME").show(2) will provide  the same output. Replace df with name of your Spark DataFrame.

2. Create a new DataFrame using expr to get only those rows where STATUS is "S"
Note that expr() just perfoms the operation, it doesnt filter our the rows which evaluate to false.
2. Sort DataFrame: use the col function to sort the DataFrame on "SID" column

**EXERCISE-4: USING THE HH_DATA, TRY OUT THESE COMMON OPERATIONS ON A DATAFRAME**
1. Using the ```filter()``` function create a DataFrame of people who are 5 years and younger?
2. Find **household size** using ```groupby()``` on the hh_id column and appropriate function (e.g., count, avg)
3. Add the household size to the ```sdf_hh``` dataframe
4. Rename columns as follows: ```P08-->age```, ```P07M-->dob_month``` and ```P07A-->dob_yr```
5. Find how many null values are there in the age column and drop all nulls in the DataFrame

In [5]:
# Load the dataframe
sdf_hh = spark.read.csv(str(HH_DATA),header=True, sep="\t")

In [6]:
sdf_hh.show(5)

23/02/14 14:10:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
 Schema: _c0, urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
Expected: _c0 but found: 
CSV file: file:///Users/dmatekenya/TEACHING-LOCAL/BDA-with-Python/DATA/raw/hh_data.txt
+---+-----------+--------------------+---+---+----+----+---+---+----+---+---+---+
|_c0|urban_rural|               hh_id|P03|P05|P07M|P07A|P08|P21|P22N|P23|P28|P29|
+---+-----------+--------------------+---+---+----+----+---+---+----+---+---+---+
|  0|          1|11101101010011066...|  0|  1|  10|1954| 63|  1|   3|  6|  2| 24|
|  1|          1|11101101010011066...|  1|  2|   8|1950| 67|  1|   3|  6|  2| 28|
|  2|          1|11101101010011066...|  2|  1|   3|1980| 38|  1|   5|  1|  1|   |
|  3|          1|11101101010011066...|  2|  1|   9|1984| 33|  1|   5|  2|  1|   |
|  4|          1|11101101010011066...|  0|  1|  12|1984| 33|

In [10]:
# Question-1
# Get dataframe for undr 5 years
sdf_hh2 = sdf_hh.withColumnRenamed('P08', 'age')
sdf_hh_u5 = sdf_hh2.filter('age <= 5')
total_pop = sdf_hh.count()
cnt_u5_children = sdf_hh_u5.count()
print('There are {:,} under-five children in the country out of a total of {:,} people'.format(
    cnt_u5_children, total_pop))

There are 4,448,568 under-five children in the country out of a total of 25,674,196 people


In [15]:
# Question-2
# Create a new dataframe called sdf_hh_size which 
# gives number of persons per household by grouping by 
# hh_id
sdf_hh_size = sdf_hh2.groupby('hh_id').count()
sdf_hh_size.show(5)

+--------------------+-----+
|               hh_id|count|
+--------------------+-----+
|11101101010021060...|    7|
|11101101010031065...|    1|
|11101101010031065...|    6|
|11101101010031066...|    7|
|11101101020011036...|    3|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Add the hh_size column to the sdf_hh dataframe
# using join

In [21]:
sdf_hh3 = sdf_hh2.join(sdf_hh_size, on='hh_id')
sdf_hh3.cache()
sdf_hh3.show(5)

23/02/14 14:37:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
 Schema: _c0, urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
Expected: _c0 but found: 
CSV file: file:///Users/dmatekenya/TEACHING-LOCAL/BDA-with-Python/DATA/raw/hh_data.txt


+--------------------+----+-----------+---+---+----+----+---+---+----+---+---+---+-----+
|               hh_id| _c0|urban_rural|P03|P05|P07M|P07A|age|P21|P22N|P23|P28|P29|count|
+--------------------+----+-----------+---+---+----+----+---+---+----+---+---+---+-----+
|11101101010021060...|1719|          1|  0|  1|   5|1976| 42|  1|   5|  1|  2| 34|    7|
|11101101010021060...|1720|          1|  1|  2|  10|1985| 32|  1|   5|  1|  2| 25|    7|
|11101101010021060...|1721|          1|  2|  1|  10|2010|  7|  2|   2|  5|   |   |    7|
|11101101010021060...|1722|          1|  2|  1|   7|2013|  4|  2|   1|   |   |   |    7|
|11101101010021060...|1723|          1|  2|  1|   4|2016|  2|   |    |   |   |   |    7|
+--------------------+----+-----------+---+---+----+----+---+---+----+---+---+---+-----+
only showing top 5 rows



In [20]:
# change column name from 'count' to hh_size
sdf_hh4 = sdf_hh3.withColumnRenamed('count', 'hh_size')
sdf_hh4.show(5)

23/02/14 14:35:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
 Schema: _c0, urban_rural, hh_id, P03, P05, P07M, P07A, P08, P21, P22N, P23, P28, P29
Expected: _c0 but found: 
CSV file: file:///Users/dmatekenya/TEACHING-LOCAL/BDA-with-Python/DATA/raw/hh_data.txt


+--------------------+---+-----------+---+---+----+----+---+---+----+---+---+---+-------+
|               hh_id|_c0|urban_rural|P03|P05|P07M|P07A|age|P21|P22N|P23|P28|P29|hh_size|
+--------------------+---+-----------+---+---+----+----+---+---+----+---+---+---+-------+
|11101101010011066...| 48|          1|  0|  1|   2|1988| 30|  1|   3|  1|  2| 24|      4|
|11101101010011066...| 49|          1|  1|  2|   5|1991| 27|  1|   3|  4|  2| 21|      4|
|11101101010011066...| 50|          1|  2|  2|   2|2013|  5|  2|   1|  5|   |   |      4|
|11101101010011066...| 51|          1|  3|  2|   1|1948| 70|  1|   3|  6|  4| 22|      4|
|11101101010011066...|191|          1|  0|  2|   2|1961| 57|  1|   3|  1|  2| 26|      5|
+--------------------+---+-----------+---+---+----+----+---+---+----+---+---+---+-------+
only showing top 5 rows



In [23]:
# Question-: Rename columns as follows: 
# P08-->age, P07M-->dob_month and P07A-->dob_yr
sdf_hh5 = (sdf_hh4
           .withColumnRenamed('P07M',  'dob_month')
           .withColumnRenamed('P07A',  'dob_yr'))
sdf_hh5.show(5)

+--------------------+----+-----------+---+---+---------+------+---+---+----+---+---+---+-------+
|               hh_id| _c0|urban_rural|P03|P05|dob_month|dob_yr|age|P21|P22N|P23|P28|P29|hh_size|
+--------------------+----+-----------+---+---+---------+------+---+---+----+---+---+---+-------+
|11101101010021060...|1719|          1|  0|  1|        5|  1976| 42|  1|   5|  1|  2| 34|      7|
|11101101010021060...|1720|          1|  1|  2|       10|  1985| 32|  1|   5|  1|  2| 25|      7|
|11101101010021060...|1721|          1|  2|  1|       10|  2010|  7|  2|   2|  5|   |   |      7|
|11101101010021060...|1722|          1|  2|  1|        7|  2013|  4|  2|   1|   |   |   |      7|
|11101101010021060...|1723|          1|  2|  1|        4|  2016|  2|   |    |   |   |   |      7|
+--------------------+----+-----------+---+---+---------+------+---+---+----+---+---+---+-------+
only showing top 5 rows



## Spark-SQL functions
As usual refer to the Pyspark API documentation for an overview of spark modules and core classes. So far, we have been 
interacting with spark-SQL. Within the spark-SQL, we have worked with the following:
- DataFrame API
- Row
- Spark Session API
- Functions

For functions, we have only used simple functions such as ```count``` and ```avg``` used in aggregation. Now, lets look at user defined functions (udf). Although new and more functions are continuously being added to the spark APIs, you will often find yourself in a situation where you need to define a small Python function to apply on spark DataFrame or other data types in spark.

In [ ]:
Image("../DOCS/images/spark-functions.png", width=700)

### Creating and using user defined functions
Spark provides two types of user defined functions (UDF):
1. [Simple UDF](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.udf.html#pyspark.sql.functions.udf).
We use the ```udf``` keyword to define a UDF using lambda. When using the ```def``` keyword, we use ```udf``` as a decorator. When defining UDF, specifying the Spark Data type to be returned is important though in some cases (e.g., for strings) you can getaway without specifying the data type. Once defined, you can call the UDF by using ```col()``` to select a single column or multiple columns as input into the UDF.
2. [Pandas UDF](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pandas_udf.html#pyspark.sql.functions.pandas_udf). With pandas on Spark API now availablee, not sure this function is useful anymore but it probably is. Esseentially, you use pandas functitons on a small slice of a Spark DataFrame.

# How can we use user defined functions in this dataframe?
For this ```sdf``` dataframe, I want to know the earliest activity date. For that, I need to convert ```ACTIVITY_TIME``` from String to Spark ```DateType``` or ```TimestampType```

In [24]:
sdf_act = spark.read.csv(str(ACTIV_LOG), header=True)
sdf_act.show(5)

+---+-----------+--------------------+------+
|SID|ACTIVITY_ID|       ACTIVITY_TIME|STATUS|
+---+-----------+--------------------+------+
|584|       1291|13-APR-15 10.33.4...|     S|
|584|       1286|13-APR-15 10.33.4...|     S|
|584|       1285|13-APR-15 10.33.4...|     S|
|584|       1284|13-APR-15 10.33.4...|     S|
|584|       1288|13-APR-15 10.33.4...|     S|
+---+-----------+--------------------+------+
only showing top 5 rows



In [31]:
# Use Spark UDF to add date and datetime with lambda setup
# Note that in this case, the return data typee is Spark TimestampType(), DateType()
# date_fmt1='%Y%m%d%H%M%S'

# For dateitme, you need to provide the datetime format of the string
# please see here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
# for more details on converting from datetime string 
# to datetime Python object
date_fmt2 = "%d-%b-%y %H.%M.%S"

# define UDF
add_datetime = udf(lambda x: datetime.strptime(x[:-13], date_fmt2), TimestampType())
add_date = udf(lambda x: datetime.strptime(x[:-13], date_fmt2), DateType())

In [33]:
# Add datetime and date column with our functions above
sdf_act2 = sdf_act.withColumn('datetime', add_datetime(col('ACTIVITY_TIME')))
sdf_act3 = sdf_act2.withColumn('date', add_date(col('ACTIVITY_TIME')))
sdf_act3.show(5)

+---+-----------+--------------------+------+-------------------+----------+
|SID|ACTIVITY_ID|       ACTIVITY_TIME|STATUS|           datetime|      date|
+---+-----------+--------------------+------+-------------------+----------+
|584|       1291|13-APR-15 10.33.4...|     S|2015-04-13 10:33:42|2015-04-13|
|584|       1286|13-APR-15 10.33.4...|     S|2015-04-13 10:33:42|2015-04-13|
|584|       1285|13-APR-15 10.33.4...|     S|2015-04-13 10:33:42|2015-04-13|
|584|       1284|13-APR-15 10.33.4...|     S|2015-04-13 10:33:42|2015-04-13|
|584|       1288|13-APR-15 10.33.4...|     S|2015-04-13 10:33:42|2015-04-13|
+---+-----------+--------------------+------+-------------------+----------+
only showing top 5 rows



In [36]:
# Sort by date find the earliest and oldest
sdf_act3_sorted = sdf_act3.sort(col('date'))
earliest = sdf_act3_sorted.head()

ERROR:root:KeyboardInterrupt while sending command.               (0 + 12) / 63]
Traceback (most recent call last):
  File "/Users/dmatekenya/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/dmatekenya/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/dmatekenya/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

23/02/15 05:38:39 ERROR Executor: Exception in task 34.0 in stage 46.0 (TID 244)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/folders/w3/dfvgjkh10wz8t573m_c20xf00000gp/T/ipykernel_73734/873190141.py", line 12, in <lambda>
TypeError: 'NoneType' object is not subscriptable

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:

460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.convert.Wrappers$IteratorWrapper.hasNext(Wrappers.scala:32)
	at org.sparkproject.guava.collect.Ordering.leastOf(Ordering.java:628)
	at org.apache.spark.util.collection.Utils$.takeOrdered(Utils.scala:37)
	at org.apache.spark.rdd.RDD.$anonfun$takeOrdered$2(RDD.scala:1539)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2$adapted(RDD.scala:855)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd

23/02/15 05:38:40 WARN TaskSetManager: Lost task 32.0 in stage 46.0 (TID 242) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)
23/02/15 05:38:40 WARN TaskSetManager: Lost task 33.0 in stage 46.0 (TID 243) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)
23/02/15 05:38:40 WARN TaskSetManager: Lost task 31.0 in stage 46.0 (TID 241) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)


23/02/15 05:38:40 WARN TaskSetManager: Lost task 28.0 in stage 46.0 (TID 238) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)
23/02/15 05:38:40 WARN TaskSetManager: Lost task 25.0 in stage 46.0 (TID 235) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)


23/02/15 05:38:40 WARN TaskSetManager: Lost task 24.0 in stage 46.0 (TID 234) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)


23/02/15 05:38:41 WARN TaskSetManager: Lost task 29.0 in stage 46.0 (TID 239) (172.29.175.158 executor driver): TaskKilled (Stage cancelled)


In [ ]:
sdf_act3.sort?

In [ ]:
# Exercise: Try outt UDF on the activity_log_raw file
sdf = spark.read.csv(CSV_PATH, header=True)

**EXERCISE-5: AGE FROM DOB**

Use the hh_data.txt to do the following:
1. create a column called ```age_from_dob``` which calculates age from the dob_yr column.
2. Drop all observations where ```age_from_dob``` and ```age``` dont match